In [1]:
from pdbfixer import PDBFixer
from openmm.app import PDBFile
import ase

In [2]:
# (1) Input FASTA sequence of peptide and convert to pdb 

fasta = "LILKPLKLLKCLKKL"

import os 
os.system('chmod +x fasta_to_pdb.sh')
os.system(f'./fasta_to_pdb.sh {fasta}')

 PyMOL(TM) Molecular Graphics System, Version 2.5.0.
 Copyright (c) Schrodinger, LLC.
 All Rights Reserved.
 
    Created by Warren L. DeLano, Ph.D. 
 
    PyMOL is user-supported open-source software.  Although some versions
    are freely available, PyMOL is not in the public domain.
 
    If PyMOL is helpful in your work or study, then please volunteer 
    support for our ongoing efforts to create open and affordable scientific
    software by purchasing a PyMOL Maintenance and/or Support subscription.

    More information can be found at "http://www.pymol.org".
 
    Enter "help" for a list of commands.
    Enter "help <command-name>" for information on a specific command.

 Hit ESC anytime to toggle between text and graphics.

 Detected 48 CPU cores.  Enabled multithreaded rendering.
PyMOL>fab BLILKPLKLLKCLKKLZ; save LILKPLKLLKCLKKL.pdb
 Save: wrote "LILKPLKLLKCLKKL.pdb".


0

In [ ]:
# (2) Fix PDB file - is this necessary ?

filename = f'{fasta}.pdb'
fixer = PDBFixer(filename)
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(pH=7.0)


with open(f'{fasta}_fixed.pdb', 'w') as f:
    PDBFile.writeFile(fixer.topology, fixer.positions, f)

In [ ]:
## PERFORM GEOMETRY OPTIMISATION ## 
# XTB ? #

In [55]:
# (4) Geometry optimisation of peptide structure 
# (5) Box created of four copies of peptide
# (6) Run simulation on peptide box in xTB
# (7) Process output, generate COLVAR file 
# (8) Train neural network on COLVAR data 
# (9) Run DFTB+ simulations using ML model 
# (10) 

import numpy as np
coords = peptide.get_positions()  # Nx3 array of atomic coordinates
center = peptide.get_center_of_mass()  # or geometric center

distances = np.linalg.norm(coords - center, axis=1)
r_peptide = distances.max()

r_peptide

np.float64(23.644648232119852)

In [6]:
from ase.io import read,write
import math
import random
import numpy as np

peptide_number = 4
radius = 60
margin = 5

peptide = read(f"{fasta}.pdb")

combined_peptides = peptide.copy()

for i in range(1, peptide_number):
    new_peptide = peptide.copy()
    
    # Random rotation angle in degrees (because the example uses degrees)
    rot_angle = random.uniform(0, 360)
    
    # Pick a random axis from x, y, z (or their negatives)
    axis = random.choice(['x', 'y', 'z', '-x', '-y', '-z'])
    
    # Apply rotation around chosen axis and angle
    new_peptide.rotate(rot_angle, axis)
    
    # Position on circle
    angle = 2 * math.pi * i / peptide_number
    x = radius * math.cos(angle)
    y = radius * math.sin(angle)
    z = 0
    
    new_peptide.translate([x, y, z])
    combined_peptides += new_peptide


coords = peptide.get_positions()
center = peptide.get_center_of_mass()
distances = np.linalg.norm(coords - center, axis=1)
r_peptide = distances.max()

# Compute cell size (large enough for circle + peptide size + margin)
cell_size = 2 * (radius + r_peptide) + margin

combined_peptides.set_cell([cell_size, cell_size, cell_size])
combined_peptides.center()

ase.io.write(f'4_{fasta}.pdb', combined_peptides)

print(f"Cell size: {cell_size:.2f} Å")

Cell size: 181.88 Å
